# 8.) Plotting good crystal labels training data in napari

In [1]:
import napari
from napari_clusters_plotter._measure import MeasureWidget
import pandas as pd
from os import listdir
from os.path import isfile, join
from napari_time_slicer._function import convert_to_2d_timelapse
from skimage.segmentation import relabel_sequential
import numpy as np
from skimage import measure
import os

In [2]:
#change label number in table
table = pd.read_csv("Trainingdata_good_crystals.csv")
table.loc[:,"label"] = np.arange(1,table.shape[0]+1)
table
table.to_csv("Measurements_relabeled_labels.csv") 

In [3]:
#get image folder and good crystal labels
image_loc = "/Users/mara/Desktop/Bachelorarbeit/Robert - PoL/20220523_training data/1. images/"
label_loc = "/Users/mara/Desktop/Bachelorarbeit/Robert - PoL/20220523_training data/5. good crystal labels/"

#append directory and sort
def get_sorted_image_list(directory):
    files = [join(directory, f) for f in listdir(directory) if isfile(join(directory, f)) and f.endswith('tif')]
    return sorted(files)
all_image_locs = get_sorted_image_list(image_loc)
all_label_locs = get_sorted_image_list(label_loc)

#open napari 
viewer = napari.Viewer()
viewer.open(all_image_locs,stack=True)
viewer.open(all_label_locs,stack=True, layer_type = "labels")
viewer.layers[0].name = "images"
viewer.layers[1].name = "labels"

#convert folders to 2D timelaps --> behaves like a stack
image_layer = convert_to_2d_timelapse(viewer.layers["images"],viewer)
label_layer = convert_to_2d_timelapse(viewer.layers["labels"],viewer)

timelapse_label_image = np.asarray(label_layer.data)

# relabel image 
relabeled_timelapse = []
relabeled_image = np.zeros((1,))
for i in range(timelapse_label_image.shape[0]):
    relabeled_image,forward_map,inverse_map = relabel_sequential(timelapse_label_image[i], offset=relabeled_image.max()+1)
    relabeled_timelapse.append(relabeled_image)

relabeled_timelapse = np.asarray(relabeled_timelapse)

#append images and labels to napari
viewer.layers.append(image_layer)
relabelled_layer = viewer.add_labels(relabeled_timelapse)
viewer.layers.pop("images")
viewer.layers.pop("labels")

#open table
widget = MeasureWidget(viewer)
widget.run(
    image_layer =image_layer,
    labels_layer=relabelled_layer,
    region_props_source="Upload file",
    reg_props_file ="Measurements_relabeled_labels.csv",
    n_closest_points_str="[2,3,4]",
)

/Users/mara/opt/anaconda3/envs/devbio-napari-env/lib/python3.9/site-packages/napari_tools_menu/__init__.py:168: FutureWarning: Public access to Window.qt_viewer is deprecated and will be removed in
v0.5.0. It is considered an "implementation detail" of the napari
application, not part of the napari viewer model. If your use case
requires access to qt_viewer, please open an issue to discuss.
  self.tools_menu = ToolsMenu(self, self.qt_viewer.viewer)


Measurement running
Region properties source: Upload file
